# Assistants

Sabemos que los LLMs son grandes cajas negras que interpretan el lenguaje natural y nos pueden dar respuestas tambien en nuestro lenguaje de acuerdo a la informacion que tienen comprimida adentro de acuerdo a su entrenamiento.

Que pasa si a estos LLMs ademas les damos herramientas para que puedan obtener informacion actualizada y verificada del mundo real?

## OpenAI Functions

Veamos como podemos darles estas herramientas a los LLMs. En OpenAI les llaman functions:

In [ ]:
!pip install openai --upgrade --quiet

In [ ]:
import openai

api_key="INGRESA AQUI TU TOKEN DE OPENAI"

In [ ]:
#@title [Opcional] cargar el token desde colab para que no se vea al dar clases {display-mode:"form"}
if api_key == "INGRESA AQUI TU TOKEN DE OPENAI":
    from google.colab import userdata
    api_key = userdata.get('OPENAI_TOKEN')
    openai.api_key = api_key

### OpenAI tools:

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="celcius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "26",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# define a tool
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_tokens=100,
    temperature=0.5,
)

In [ ]:
response

In [ ]:
print("Function to call:",response.choices[0].message.tool_calls[0].function.name)
print("Function arguments:",response.choices[0].message.tool_calls[0].function.arguments)

In [ ]:
messages.append(response.choices[0].message)

In [ ]:
# Step 2: determine if the response from the model includes a tool call.   
tool_calls = response.choices[0].message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_function_arguments = eval(tool_calls[0].function.arguments)
    
    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'get_current_weather':
        results = get_current_weather(**tool_function_arguments)
        
        messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = openai.chat.completions.create(
            model="gpt-4",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    print(response.content) 

## Referencias:

- https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)